# <center style="color: #003b7f"> From Publication to Citation: Understanding the Factors <br/> Driving Research Recognition  </center> <a class='tocSkip'>
    
**MSDS 2023 Term 4 SLT1** | Loraine Menorca, BJ Enrik Yepes, Gregory del Carmen, Rozz Banquerigo

The code below was used to extract the data we needed for our EDA / Descriptive analytics. The strategy was to write the files into our S3 buckets and transfer them between the members for quick access to the files. All queries below after fetching the data will be in the S3 bucket as a CSV file.
    
**! Note:** This was run using PySpark via AWS EMR studio. Rerunning the entire notebook outside a similar environment is not advisable.
    
***

In [6]:
import boto3

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [7]:
aws_access_key_id = ''
aws_secret_access_key = ''

# Create session for s3
session = boto3.Session(aws_access_key_id=aws_access_key_id, aws_secret_access_key=aws_secret_access_key)
s3 = session.client('s3')


VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [13]:
import pyspark.sql.functions as F
import pandas as pd
from pyspark.sql.window import Window
from pyspark.sql.functions import row_number

# Bucket used by the team for analysis.
dir_ = 's3bucketemr'

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [11]:
df_lim = spark.read.parquet('s3a://s3bucketemr/crossref.parquet/')
# df_lim = spark.read.parquet('s3a://s3bucketemr/crossref.parquet/part-00065-d03a3643-77f3-4f99-bf3e-13f9c2ec6505-c000.snappy.parquet')
# df_lim = df_lim.limit(5000).cache()


VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [10]:
df_lim.printSchema()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- col: struct (nullable = true)
 |    |-- DOI: string (nullable = true)
 |    |-- ISBN: array (nullable = true)
 |    |    |-- element: string (containsNull = true)
 |    |-- ISSN: array (nullable = true)
 |    |    |-- element: string (containsNull = true)
 |    |-- URL: string (nullable = true)
 |    |-- abstract: string (nullable = true)
 |    |-- accepted: struct (nullable = true)
 |    |    |-- date-parts: array (nullable = true)
 |    |    |    |-- element: array (containsNull = true)
 |    |    |    |    |-- element: long (containsNull = true)
 |    |-- alternative-id: array (nullable = true)
 |    |    |-- element: string (containsNull = true)
 |    |-- approved: struct (nullable = true)
 |    |    |-- date-parts: array (nullable = true)
 |    |    |    |-- element: array (containsNull = true)
 |    |    |    |    |-- element: long (containsNull = true)
 |    |-- archive: array (nullable = true)
 |    |    |-- element: string (containsNull = true)
 |    |-- article-numb

## Top Article Based on Reference Count

In [ ]:
import pandas as pd

df_ref = (df_lim.withColumn('title_1', F.col('col.title').getItem(0))
                .withColumn('subj', F.concat_ws(', ', F.col('col.subject')))
                .withColumn('year', F.when(F.size(F.element_at(F.col('col.published.date-parts'), 1)) >= 1, 
                                           F.element_at(F.col('col.published.date-parts'), 1)[0]))
                .withColumn('month', F.when(F.size(F.element_at(F.col('col.published.date-parts'), 1)) >= 2,
                                            F.element_at(F.col('col.published.date-parts'), 1)[1]))
                .withColumn('day', F.when(F.size(F.element_at(F.col('col.published.date-parts'), 1)) >= 3, 
                                          F.element_at(F.col('col.published.date-parts'), 1)[2]))
                .select('title_1', 'col.is-referenced-by-count', 'subj', 'year', 'month', 'day')
                .orderBy(F.desc('is-referenced-by-count'))
                .where(F.col('is-referenced-by-count') > 0)
                .toPandas())

# Convert float to integer
df_ref['year'] = pd.to_numeric(df_ref['year'], errors='coerce').astype('Int64')
df_ref['month'] = pd.to_numeric(df_ref['month'], errors='coerce').astype('Int64')
df_ref['day'] = pd.to_numeric(df_ref['day'], errors='coerce').astype('Int64')

write_csv = df_ref.to_csv(index=False)

s3.put_object(Body=write_csv, Bucket=dir_, Key='extracted_csv/crossref_rec_count.csv')

df_ref.head(10)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

## Top 10 Article Per Year Based on Reference Count (Top N can be changed just chang N variable)

In [16]:
df_top10_ref = (df_lim.withColumn('title_1', F.col('col.title').getItem(0))
                    .withColumn('subj', F.concat_ws(', ', F.col('col.subject')))
                    .withColumn('year', F.when(F.size(F.element_at(F.col('col.published.date-parts'), 1)) >= 1, 
                                               F.element_at(F.col('col.published.date-parts'), 1)[0]))
                    .withColumn('month', F.when(F.size(F.element_at(F.col('col.published.date-parts'), 1)) >= 2,
                                                F.element_at(F.col('col.published.date-parts'), 1)[1]))
                    .withColumn('day', F.when(F.size(F.element_at(F.col('col.published.date-parts'), 1)) >= 3, 
                                              F.element_at(F.col('col.published.date-parts'), 1)[2]))
                    .select('title_1', 'col.is-referenced-by-count', 'subj', 'year', 'month', 'day')
                    .orderBy(F.desc('is-referenced-by-count'))
                    .where(F.col('is-referenced-by-count') > 0))


window_spec = Window.partitionBy("year").orderBy(df_top10_ref["is-referenced-by-count"].desc())

df_fil = df_top10_ref.withColumn("row_num", row_number().over(window_spec))


# Filter the DataFrame to get the top 10 rows for each year
N = 10
df_top10_ref_fin = df_fil.filter(df_fil.row_num <= N).toPandas()

# Show the result
df_top10_ref_fin['year'] = pd.to_numeric(df_top10_ref_fin['year'], errors='coerce').astype('Int64')
df_top10_ref_fin['month'] = pd.to_numeric(df_top10_ref_fin['month'], errors='coerce').astype('Int64')
df_top10_ref_fin['day'] = pd.to_numeric(df_top10_ref_fin['day'], errors='coerce').astype('Int64')

write_csv = df_top10_ref_fin.to_csv(index=False)

s3.put_object(Body=write_csv, Bucket=dir_, Key='extracted_csv/crossref_publish_ref_top10_yearly_count.csv')

df_top10_ref_fin.head(10)


VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

                                             title_1  ...  row_num
0        The Journal of Sports and Entertainment Law  ...        1
1       Specification for Structural Steel for Ships  ...        2
2  Figure 10: Global function (<tt>s(log(conc))</...  ...        3
3  Devesa de l'Albufera: el cambio de paradigma e...  ...        4
4                               Karakter og handling  ...        1
5                           Filmteori og filmanalyse  ...        1
6  Begrebet diakroni inden for litteraturforskningen  ...        1
7                       Skitse til en handlingsmodel  ...        2
8                         Udviskning af en metafysik  ...        3
9               Strukturalistisk litteraturvidenskab  ...        1

[10 rows x 7 columns]

## Top Subjects Count

In [ ]:
df_subj = (df_lim.where(F.col('col.subject').isNotNull())
                 .withColumn('subject', F.explode(df_lim.col.subject))
                 .groupby('subject').count().orderBy(F.desc('count')).toPandas())

write_csv = df_subj.to_csv(index=False)

s3.put_object(Body=write_csv, Bucket=dir_, Key='extracted_csv/crossref_subject_count.csv')

df_subj.head(10)


## Top 10 Subjects Per Year Based on Journal Count (Top N can be changed just chang N variable)

In [ ]:
df_top10_subj = (df_lim.where(F.col('col.subject').isNotNull())
                      .withColumn('subject', F.explode(df_lim.col.subject))
                      .withColumn('year', F.when(F.size(F.element_at(F.col('col.published.date-parts'), 1)) >= 1, 
                                               F.element_at(F.col('col.published.date-parts'), 1)[0]))
                      .select('subject', 'year')
                      .where(F.col('year').isNotNull())
                      .groupBy('subject', 'year').count())

window_spec = Window.partitionBy("year").orderBy(df_top10_subj["count"].desc())

df_fil = df_top10_subj.withColumn("row_num", row_number().over(window_spec))

# Filter the DataFrame to get the top 10 rows for each year
N = 10
df_top10_subj_fin = df_fil.filter(df_fil.row_num <= N).toPandas()

# Show the result
df_top10_subj_fin['year'] = pd.to_numeric(df_top10_subj_fin['year'], errors='coerce').astype('Int64')


write_csv = df_top10_subj_fin.to_csv(index=False)

s3.put_object(Body=write_csv, Bucket=dir_, Key='extracted_csv/crossref_publish_subj_top10_yearly_count.csv')

df_top10_subj_fin.head(10)


## Publication Counts Per Year

In [ ]:
df_published = (df_lim.withColumn("ExplodedDate", F.explode(F.col("col.published.date-parts")))
                      .withColumn("year", F.col("ExplodedDate").getItem(0))
                      .filter(F.col("year").isNotNull())
                      .groupBy("year").count()
                      .withColumnRenamed("count", "count_published")
                      .orderBy("year").toPandas())


write_csv = df_published.to_csv(index=False)

s3.put_object(Body=write_csv, Bucket=dir_, Key='extracted_csv/crossref_publish_count.csv')

df_published.head(10)

## Data Science Related Topics Per Year

In [ ]:
df_ds = (df_lim.where(F.col('col.subject').isNotNull())
               .withColumn('subject', F.explode(df_lim.col.subject))
               .withColumn('year', F.when(F.size(F.element_at(F.col('col.published.date-parts'), 1)) >= 1, 
                                           F.element_at(F.col('col.published.date-parts'), 1)[0]))
               .select('subject', 'year')
               .where(F.lower(F.col('subject')).like('%machine learning%') |
                     F.lower(F.col('subject')).like('%data science%') |
                     F.lower(F.col('subject')).like('%data analytics%') |
                     F.lower(F.col('subject')).like('%artificial intelligence%') |
                     F.lower(F.col('subject')).like('%a.i.%'))
               .groupBy('year').count().orderBy(F.col('year')).toPandas()
           )

write_csv = df_ds.to_csv(index=False)
df_ds['year'] = pd.to_numeric(df_ds['year'], errors='coerce').astype('Int64')

s3.put_object(Body=write_csv, Bucket=dir_, Key='extracted_csv/yearly_data_science_count.csv')

df_ds.head(10)

## Data Science Related Topics Per Year Reference Count

In [ ]:
df_ds = (df_lim.where(F.col('col.subject').isNotNull())
               .withColumn('subject', F.explode(df_lim.col.subject))
               .withColumn('year', F.when(F.size(F.element_at(F.col('col.published.date-parts'), 1)) >= 1, 
                                           F.element_at(F.col('col.published.date-parts'), 1)[0]))
               .select('subject', 'year', 'col.is-referenced-by-count')
               .where(F.lower(F.col('subject')).like('%machine learning%') |
                     F.lower(F.col('subject')).like('%data science%') |
                     F.lower(F.col('subject')).like('%data analytics%') |
                     F.lower(F.col('subject')).like('%artificial intelligence%') |
                     F.lower(F.col('subject')).like('%a.i.%'))
               .groupBy('year').agg(F.sum(F.col('is-referenced-by-count'))).orderBy(F.col('year')).toPandas()
           )

write_csv = df_ds.to_csv(index=False)
df_ds['year'] = pd.to_numeric(df_ds['year'], errors='coerce').astype('Int64')

s3.put_object(Body=write_csv, Bucket=dir_, Key='extracted_csv/yearly_data_science_ref_count.csv')

df_ds.head(10)

## Subject Reference Count

In [ ]:
df_subj = (df_lim.where(F.col('col.subject').isNotNull())
                 .withColumn('subject', F.explode(df_lim.col.subject))
                 .groupby('subject').agg(F.sum(F.col('col.is-referenced-by-count')).alias('ref_count'))
                 .orderBy(F.desc('ref_count')).toPandas())

write_csv = df_subj.to_csv(index=False)

s3.put_object(Body=write_csv, Bucket=dir_, Key='extracted_csv/subject_reference_count.csv.csv')

df_subj.head(10)


## Top Funder Count

In [ ]:
df_fund = (df_lim.where(F.col('col.funder').isNotNull())
                 .withColumn('funder_elem', F.explode(df_lim.col.funder))
                 .withColumn('funder', F.col('funder_elem.name'))
                 .groupby('funder').count()
                 .orderBy(F.desc('count')).toPandas())

write_csv = df_fund.to_csv(index=False)

s3.put_object(Body=write_csv, Bucket=dir_, Key='extracted_csv/top_fund_count.csv')

df_fund.head(10)

## Top Places Count

In [ ]:
df_fund = (df_lim.where(F.col('col.institution').isNotNull())
                 .withColumn('institution_elem', F.explode(df_lim.col.institution))
                 .withColumn('place', F.explode(F.col('institution_elem.place')))
                 .groupby('place').count()
                 .orderBy(F.desc('count')).toPandas())

write_csv = df_fund.to_csv(index=False)

s3.put_object(Body=write_csv, Bucket=dir_, Key='extracted_csv/place_count.csv')

df_fund.head(10)